# EXAMPLE: Import Data from Google Sheets

This notebook will walkthrough getting you integrated with Google's Sheets API so you can pull data from Google Sheets into a data frame and use it accordingly. 

In this case, we are using a Google Sheet document created by IFTTT of YouTube Liked Videos.

-----

### Installation and Setup

- Go to https://developers.google.com/sheets/api/quickstart/python.
- Click "ENABLE THE GOOGLE SHEETS API" and go through additional steps. 
- Configure below with sheet id and name. 
- Run Notebook

NOTE: This should prompt you to open a URL to confirm access with Google. The end result should be a new file created called "token.pickle"

-----

### Libraries

In [1]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from datetime import date, datetime as dt, timedelta as td
import pandas as pd

In [2]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

### The Sheet ID and target range (i.e. sheet name) of a sample spreadsheet.

* Copy Sheet ID from URL of Targetted Sheet
* Add ID to Configuration below as well as sheet name and/or range. 

In [3]:
# Configure to Your Specific Sheet 

# SAMPLE_SPREADSHEET_ID = '1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms'
# SAMPLE_RANGE_NAME = 'Class Data!A2:E'

SPREADSHEET_ID = "ADD_SHEET_ID_HERE"
RANGE_NAME = "Sheet1"

In [4]:
# Authentication
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

# build service
service = build('sheets', 'v4', credentials=creds)

In [5]:
# Get Data from Google Sheet and Return a List
def get_gsheet_data(SPREADSHEET_ID, RANGE_NAME):
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=RANGE_NAME).execute()
    values = result.get('values', [])
    return values

In [19]:
# get data
data_list = get_gsheet_data(SPREADSHEET_ID, RANGE_NAME)
# type(data) #list

In [45]:
# create data and frame and use first row as header
data = pd.DataFrame(data_list)
new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

In [46]:
# export to csv
data.to_csv("data/youtube_liked_videos.csv", index=False)